# Light exposure analysis with pyActigraphy

![Light&Gears](img/daniele-levis-pelusi-Pp9qkEV_xPk-unsplash.jpg)

Photo by <a href="https://unsplash.com/@yogidan2012?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Daniele Levis Pelusi</a> on <a href="https://unsplash.com/s/photos/clocks-light?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

## Disclaimer

The development of the pyActigraphy module for analysing light exposure data was led and financially supported by members of the Daylight Academy Project *The role of daylight for humans* (led by Mirjam Münch, Manuel Spitschan). The module is part of the Human Light Exposure Database. For more information about the project, please see
https://daylight.academy/projects/state-of-light-in-humans/.

## Introduction

Similarly to the analysis of locomotor activity via actigraphy, light exposure data analysis is difficult because of the lack of open-source analysis softwares that gives users access to a list of various analysis metrics.

The light exposure data analysis module of *pyActigraphy* is meant to fix this issue.

In this tutorial, we will review the light exposure analysis metrics currently available in pyActigraphy:

* Exposure levels
* Summary statistics
* Time above threshold (TAT)
* Mean light timing above threshold (MLit)
* Extrema (min. and max.)
* L5 and M10 (LMX)
* IS and IV
* Direct access to raw or thresholded data

## Imports

As usual, first import the necessary packages:

In [ ]:
import pyActigraphy

In [ ]:
import plotly.graph_objects as go

In [ ]:
import numpy as np

In [ ]:
import os

## Log-transformation of light data

Light data are (log10+1)-transformed in *pyActigraphy*. Therefore, when threshold values are meant to applied to the light levels, the corresponding value on the (log10+1) scale should be applied.

**NB**: an offset of 1 is added to the light data before log10 transformation in order to avoid a divergence of the log10 function when the light data values are zero: $\log_{10}(0) = -\infty $.

So, on a (log10+1) scale:

* 10 lux threshold correspond to a value of log(10+1)~1
* 100 lux threshold correspond to a value of log(100+1)~2
* 1000 lux threshold correspond to a value of log(1000+1)~3
* ...

To simply get the exact value, use the `np.log10` function of the *numpy* package we imported earlier:

In [ ]:
np.log10(10+1), np.log10(100+1), np.log10(1000+1)

## Input data

Set up a path to your favorite data file. Here, we use a recording made with a Actiwatch Spectrum Pro from Respironic:

In [ ]:
fpath = os.path.join(
    os.path.dirname(pyActigraphy.__file__),
    'tests','data', 'test_sample_rpx_ger_with_light.csv'
)

To read such a file, use the corresponding reader function:

In [ ]:
raw = pyActigraphy.io.read_raw_rpx(
    fpath,
    language='GER',
    delimiter=',',
    decimal=','
)

Now, let's verify which light channels are available for this recording:

In [ ]:
raw.light.get_channel_list()

As expected, this device records light exposure data in 4 different channels (White light + RBG).

In the following, we will learn how to analyse these different channels.

## Light exposure metrics

### Exposure level

This function returns coarse statistics (mean, median, sum, etc) about the light exposure level. It is possible to define the daily time window during which these statistics are calculated.

In [ ]:
help(raw.light.light_exposure_level)

Default settings: mean exposure level per acquisition epoch

In [ ]:
raw.light.light_exposure_level()

Median exposure level:

In [ ]:
raw.light.light_exposure_level(agg='median')

Restricting the time window to '08:00'-'16:00' on a daily basis:

In [ ]:
raw.light.light_exposure_level(
    start_time='08:00:00',
    stop_time='16:00:00',
)

It is also possible to only consider light exposure data above a certain threshold. For example, to compute the mean light exposure level above 10 lux, between '12:00' and '18:00':

In [ ]:
raw.light.light_exposure_level(
    threshold=1, # on a log10 scale
    start_time='12:00',
    stop_time='18:00'
)

### Summary statistics (mean, median, s.d., max., min., sum)

This function returns various summary statistics about the data aggregated into time bins. The length of these time bins, but also their start and stop times, are configurable.

In [ ]:
help(raw.light.summary_statistics_per_time_bin)

By default, this function returns the mean, median, sum, standard deviation, as well as the min and max, of data aggregated into consecutive 24h bins, for all light channels separately:

In [ ]:
raw.light.summary_statistics_per_time_bin()

To change the size of the time bins, now focusing on the mean light level, computed every 12h for instance:

In [ ]:
raw.light.summary_statistics_per_time_bin(bins='12h', agg_func=['mean'])

Finally, arbitrary start and stop times can be used to define the time bins;

In [ ]:
raw.light.summary_statistics_per_time_bin(
    bins=[
        ['2019-09-19 12:00:00','2019-09-19 19:59:00'],
        ['2019-09-23 12:00:00','2019-09-23 23:59:00']
    ],
    agg_func=['mean','std']
)

### Time above threshold (TAT)

This function computes the total time spent with a light exposure level above a certain threshold. Similarly to the light exposure level function, it is possible to configure the daily time window during which this metric is calculated. Various time output format are available.

In [ ]:
help(raw.light.TAT)

With the default settings (no threshold, no time window restriction):

In [ ]:
raw.light.TAT()

the results are identical to the total number of epochs in the recording.

To calculate the number of epochs spent above a threshold of 10 lux:

In [ ]:
raw.light.TAT(threshold=1)

The parameter `oformat` defines the output format. Available formats:
* 'minute':

In [ ]:
raw.light.TAT(threshold=1, oformat='minute')

* 'timedelta': 

In [ ]:
raw.light.TAT(threshold=1, oformat='timedelta')

Time spent above threshold at specific time periods:

In [ ]:
raw.light.TAT(
    threshold=1, start_time='08:00:00', stop_time='20:00:00', oformat='timedelta'
)

### Time above threshold per period (TATp)

Sometimes, it could be useful to compute the TAT on consecutive periods of time in order to assess the evolution of such metric through time or before and after a specific intervention. While it is always to possible to read the same file multiple times and restrict its start and stop time to different periods each time, *pyActigraphy* allows users to directly compute the TAT on a daily basis.

This function has the same input parameters as the TAT function:

In [ ]:
help(raw.light.TATp)

In [ ]:
raw.light.TATp(threshold=2, oformat='minute')

### Mean light timing (MLit)

This function calculates the mean light timing above threshold ($\mathrm{MLit}^{XXX}$), defined in [Reid et al. (2014)](https://doi.org/10.1371/journal.pone.0092251) as the average clock time of all aggregated data points above XXX lux. In the original paper, a threshold of 500 lux was chosen but in this function, the threshold is configurable.

The results are expressed in number of minutes since midnight, irrespective of the sampling period.

In [ ]:
help(raw.light.MLiT)

To compute the $\mathrm{MLit}^{500}$:

In [ ]:
# Do not forget that the light exposure data are converted to log10+1.
raw.light.MLiT(threshold=np.log10(500+1))

In [ ]:
# To convert this number of minutes into a time of day, simply divide it by the number of minutes per hour:
divmod(791.039116,60)

The mean light timing for white light exposure above 500 lux is around 13h11 for this recording.

Just like for the TAT, it might be interesting to compute the $\mathrm{MLit}^{XXX}$ variable on a daily basis. Again, *pyActigraphy* makes that operation easy:

In [ ]:
raw.light.MLiTp(threshold=np.log10(500+1))

### Extrema

This function simply returns information (timing and value) about the minimal or maximal light exposure data value.

In [ ]:
help(raw.light.get_light_extremum)

In [ ]:
raw.light.get_light_extremum(extremum='max')

In [ ]:
raw.light.get_light_extremum(extremum='min')

### L5 and M10 (values and timing)

The L5 (M10) variable refers to the 5(10)h-period of daily light exposure profile with the lowest(highest) light exposure levels. In this function, the length of this period can be configured. Information about the timing and the mean hourly light exposure levels are returned.

In [ ]:
help(raw.light.LMX)

In [ ]:
raw.light.LMX(length='10h',lowest=False)

It seems that the 10h long period with the highest light exposure levels starts at 8h21.

In [ ]:
raw.light.LMX(length='5h',lowest=True)

It seems that the 5h long period with the lowest light exposure levels starts at 00h00.

It is easy to visually check if these results are correct by computing the average daily profile of light exposure:

In [ ]:
dlp = raw.light.average_daily_profile(
    rsfreq='60min', cyclic=False, channel='Weißes Licht'
)

In [ ]:
dlp_fig = go.Figure(go.Scatter(x=dlp.index.astype(str),y=dlp))

In [ ]:
dlp_fig.show()

### IS & IV

The interdaily stability (IS) and intradaily variability (IV) were first defined in the context of the locomotor activity analysis, to quantify the stability of the circadian activity pattern and the fragmentation of this daily pattern, respectively.

Now, within the pyActigraphy module for light analysis, these metrics can be computed using the light exposure levels as inputs.

**NB**: the default parameter values for these functions are different from the values used by default for the IS/IV functions for activity data, directly accessible via `raw.IS()` or `raw.IV()`.

In [ ]:
help(raw.light.IS)

By default, light exposure data are not binarized before computing the IS:

In [ ]:
raw.light.IS()

However, it remains possible to do so by applying a threshold (at 100 lux for example):

In [ ]:
raw.light.IS(binarize=True, threshold=2)

In [ ]:
help(raw.light.IV)

By default, no data binarization is applied:

In [ ]:
raw.light.IV()

But, again, data binarization can easily be performed:

In [ ]:
raw.light.IV(binarize=True, threshold=1)

### Access to raw and thresholded data

Most of the analyses on light exposure can be performed using the various metrics available in the *pyActigraphy* light module. However, for the most fearless users that require full access to the light exposure time series, it is possible to directly access the raw data or simply the thresholded data:

In [ ]:
help(raw.light.VAT)

Inspect the first 5 epochs of raw light data:

In [ ]:
raw.light.data.head(5)

Or access the raw data thresholded at 100 lux:

In [ ]:
raw.light.VAT(2)

Et voilà! For now...